<a href="https://colab.research.google.com/github/Phionanamugga/Autonomous_Navigation/blob/main/ObjectDetection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pymap3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 1.5 MB/s eta 0:00:00


In [4]:
#!/usr/bin/env python3
"""
Advanced LiDAR-Camera Fusion System for Autonomous Driving
=======================================================

A production-ready implementation featuring:
- Modular, extensible architecture
- Real-time performance optimization
- Comprehensive error handling
- Extensive logging and monitoring
- Multi-threading support
- Advanced calibration and validation
- Performance benchmarking
- Clean code practices following industry standards

Author: Your Name
Date: 2025
"""

import os
import sys
import json
import time
import logging
import threading
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Union, Any
from pathlib import Path
import concurrent.futures
from contextlib import contextmanager

import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import pymap3d as pm
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
import yaml

In [5]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('fusion_system.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [6]:
@dataclass
class CalibrationData:
    """Encapsulates all calibration matrices and parameters."""
    P_rect: np.ndarray  # Projection matrix
    R_rect: np.ndarray  # Rectification matrix
    T_cam_velo: np.ndarray  # Camera to LiDAR transform
    T_velo_cam: np.ndarray  # LiDAR to camera transform
    T_cam_imu: np.ndarray  # Camera to IMU transform
    camera_intrinsics: Dict[str, float]
    distortion_coeffs: np.ndarray

    def __post_init__(self):
        """Validate calibration data."""
        self._validate_matrices()

    def _validate_matrices(self):
        """Validate calibration matrix dimensions and properties."""
        assert self.P_rect.shape == (3, 4), f"Invalid P_rect shape: {self.P_rect.shape}"
        assert self.R_rect.shape == (4, 4), f"Invalid R_rect shape: {self.R_rect.shape}"
        assert self.T_velo_cam.shape == (3, 4), f"Invalid T_velo_cam shape: {self.T_velo_cam.shape}"
        logger.info("Calibration matrices validated successfully")

In [7]:
@dataclass
class Detection:
    """Represents a single object detection."""
    bbox: np.ndarray  # [x1, y1, x2, y2]
    confidence: float
    class_id: int
    class_name: str
    center_3d: Optional[np.ndarray] = None  # [x, y, z] in camera coordinates
    distance: Optional[float] = None
    gps_coords: Optional[Tuple[float, float, float]] = None  # (lat, lon, alt)
    timestamp: float = field(default_factory=time.time)


In [8]:
@dataclass
class SensorFrame:
    """Container for synchronized sensor data."""
    timestamp: float
    image: np.ndarray
    lidar_points: np.ndarray
    gps_data: Dict[str, float]
    imu_data: Dict[str, float]
    frame_id: int


In [9]:
class PerformanceMonitor:
    """Tracks and reports system performance metrics."""

    def __init__(self):
        self.metrics = {
            'processing_times': [],
            'detection_counts': [],
            'fusion_accuracy': [],
            'memory_usage': [],
            'cpu_usage': []
        }
        self.start_time = time.time()

    @contextmanager
    def measure_time(self, operation: str):
        """Context manager to measure execution time."""
        start = time.time()
        try:
            yield
        finally:
            duration = time.time() - start
            self.metrics['processing_times'].append({
                'operation': operation,
                'duration': duration,
                'timestamp': time.time()
            })
            logger.debug(f"{operation} took {duration:.4f} seconds")

    def log_detection_count(self, count: int):
        """Log number of detections per frame."""
        self.metrics['detection_counts'].append({
            'count': count,
            'timestamp': time.time()
        })

    def get_performance_report(self) -> Dict[str, Any]:
        """Generate comprehensive performance report."""
        processing_times = [m['duration'] for m in self.metrics['processing_times']]
        return {
            'avg_processing_time': np.mean(processing_times) if processing_times else 0,
            'max_processing_time': np.max(processing_times) if processing_times else 0,
            'min_processing_time': np.min(processing_times) if processing_times else 0,
            'total_frames_processed': len(self.metrics['detection_counts']),
            'avg_detections_per_frame': np.mean([m['count'] for m in self.metrics['detection_counts']]) if self.metrics['detection_counts'] else 0,
            'uptime': time.time() - self.start_time
        }

In [10]:
class ConfigManager:
    """Manages system configuration."""

    DEFAULT_CONFIG = {
        'detection': {
            'model_name': 'yolov5s',
            'confidence_threshold': 0.25,
            'iou_threshold': 0.25,
            'max_detections': 100
        },
        'fusion': {
            'max_distance_threshold': 100.0,  # meters
            'point_cluster_eps': 0.5,
            'min_cluster_size': 3,
            'depth_filter_sigma': 2.0
        },
        'performance': {
            'use_gpu': True,
            'num_threads': 4,
            'batch_size': 1
        },
        'logging': {
            'level': 'INFO',
            'save_processed_frames': False
        }
    }

    def __init__(self, config_path: Optional[str] = None):
        self.config = self.DEFAULT_CONFIG.copy()
        if config_path and os.path.exists(config_path):
            self.load_config(config_path)

    def load_config(self, config_path: str):
        """Load configuration from YAML file."""
        try:
            with open(config_path, 'r') as f:
                user_config = yaml.safe_load(f)
                self._deep_update(self.config, user_config)
            logger.info(f"Configuration loaded from {config_path}")
        except Exception as e:
            logger.error(f"Failed to load config from {config_path}: {e}")

    def _deep_update(self, base_dict: Dict, update_dict: Dict):
        """Recursively update nested dictionary."""
        for key, value in update_dict.items():
            if isinstance(value, dict) and key in base_dict:
                self._deep_update(base_dict[key], value)
            else:
                base_dict[key] = value

    def get(self, key_path: str, default=None):
        """Get configuration value using dot notation."""
        keys = key_path.split('.')
        value = self.config
        for key in keys:
            if isinstance(value, dict) and key in value:
                value = value[key]
            else:
                return default
        return value

In [11]:
class DataLoader:
    """Handles loading and preprocessing of KITTI dataset."""

    def __init__(self, data_path: str):
        self.data_path = Path(data_path)
        self._validate_data_structure()
        self._load_file_paths()

    def _validate_data_structure(self):
        """Validate KITTI dataset structure."""
        required_dirs = ['image_02/data', 'velodyne_points/data', 'oxts/data']
        for dir_name in required_dirs:
            dir_path = self.data_path / dir_name
            if not dir_path.exists():
                raise FileNotFoundError(f"Required directory not found: {dir_path}")
        logger.info("Data structure validation passed")

    def _load_file_paths(self):
        """Load all file paths and ensure synchronization."""
        self.left_image_paths = sorted((self.data_path / 'image_02/data').glob('*.png'))
        self.lidar_paths = sorted((self.data_path / 'velodyne_points/data').glob('*.bin'))
        self.oxts_paths = sorted((self.data_path / 'oxts/data').glob('*.txt'))

        # Ensure all sensors have same number of frames
        lengths = [len(self.left_image_paths), len(self.lidar_paths), len(self.oxts_paths)]
        if not all(l == lengths[0] for l in lengths):
            logger.warning(f"Sensor data length mismatch: {lengths}")

        self.num_frames = min(lengths)
        logger.info(f"Loaded {self.num_frames} synchronized frames")

    def load_frame(self, frame_idx: int) -> SensorFrame:
        """Load synchronized sensor data for a specific frame."""
        if frame_idx >= self.num_frames:
            raise IndexError(f"Frame index {frame_idx} out of range")

        # Load image
        image = cv2.cvtColor(cv2.imread(str(self.left_image_paths[frame_idx])), cv2.COLOR_BGR2RGB)

        # Load LiDAR points
        lidar_points = np.fromfile(str(self.lidar_paths[frame_idx]), dtype=np.float32).reshape(-1, 4)

        # Load GPS/IMU data
        gps_imu_data = self._parse_oxts_data(str(self.oxts_paths[frame_idx]))

        return SensorFrame(
            timestamp=time.time(),
            image=image,
            lidar_points=lidar_points,
            gps_data=gps_imu_data['gps'],
            imu_data=gps_imu_data['imu'],
            frame_id=frame_idx
        )

    def _parse_oxts_data(self, oxts_path: str) -> Dict[str, Dict[str, float]]:
        """Parse OXTS GPS/IMU data file."""
        with open(oxts_path, 'r') as f:
            values = [float(x) for x in f.readline().strip().split()]

        return {
            'gps': {
                'lat': values[0],
                'lon': values[1],
                'alt': values[2],
                'roll': values[3],
                'pitch': values[4],
                'yaw': values[5]
            },
            'imu': {
                'vn': values[6],  # velocity north
                've': values[7],  # velocity east
                'vf': values[8],  # velocity forward
                'vl': values[9],  # velocity left
                'vu': values[10],  # velocity up
                'ax': values[11],  # acceleration x
                'ay': values[12],  # acceleration y
                'az': values[13],  # acceleration z
                'af': values[14],  # acceleration forward
                'al': values[15],  # acceleration left
                'au': values[16],  # acceleration up
                'wx': values[17],  # angular rate x
                'wy': values[18],  # angular rate y
                'wz': values[19]   # angular rate z
            }
        }

In [12]:
class CalibrationLoader:
    """Loads and manages calibration data."""

    def __init__(self, calib_dir: str):
        self.calib_dir = Path(calib_dir)

    def load_calibration(self) -> CalibrationData:
        """Load all calibration matrices."""
        try:
            # Load camera calibration
            cam_calib = self._load_camera_calibration()

            # Load LiDAR calibration
            lidar_calib = self._load_lidar_calibration()

            # Compute transformation matrices
            T_velo_cam = self._compute_velo_to_cam_transform(cam_calib, lidar_calib)
            T_cam_velo = np.linalg.inv(np.vstack([T_velo_cam, [0, 0, 0, 1]]))

            return CalibrationData(
                P_rect=cam_calib['P_rect'],
                R_rect=cam_calib['R_rect'],
                T_cam_velo=T_cam_velo,
                T_velo_cam=T_velo_cam,
                T_cam_imu=np.eye(4),  # Placeholder
                camera_intrinsics=cam_calib['intrinsics'],
                distortion_coeffs=cam_calib['distortion']
            )

        except Exception as e:
            logger.error(f"Failed to load calibration data: {e}")
            raise

    def _load_camera_calibration(self) -> Dict:
        """Load camera calibration matrices."""
        calib_file = self.calib_dir / 'calib_cam_to_cam.txt'
        with open(calib_file, 'r') as f:
            lines = f.readlines()

        # Parse projection matrix (P_rect_02)
        P_rect = np.array([float(x) for x in lines[25].strip().split()[1:]]).reshape(3, 4)

        # Parse rectification matrix (R_rect_00)
        R_rect_3x3 = np.array([float(x) for x in lines[24].strip().split()[1:]]).reshape(3, 3)
        R_rect = np.eye(4)
        R_rect[:3, :3] = R_rect_3x3

        # Extract intrinsics from projection matrix
        fx, fy = P_rect[0, 0], P_rect[1, 1]
        cx, cy = P_rect[0, 2], P_rect[1, 2]

        return {
            'P_rect': P_rect,
            'R_rect': R_rect,
            'intrinsics': {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy},
            'distortion': np.zeros(5)  # Assuming rectified images
        }

    def _load_lidar_calibration(self) -> Dict:
        """Load LiDAR calibration matrices."""
        calib_file = self.calib_dir / 'calib_velo_to_cam.txt'
        with open(calib_file, 'r') as f:
            lines = f.readlines()

        R = np.array([float(x) for x in lines[1].strip().split()[1:]]).reshape(3, 3)
        t = np.array([float(x) for x in lines[2].strip().split()[1:]]).reshape(3, 1)

        return {'R': R, 't': t}

    def _compute_velo_to_cam_transform(self, cam_calib: Dict, lidar_calib: Dict) -> np.ndarray:
        """Compute complete transformation from LiDAR to camera coordinates."""
        # LiDAR to camera coordinate system
        T_velo_cam_0 = np.hstack([lidar_calib['R'], lidar_calib['t']])
        T_velo_cam_0 = np.vstack([T_velo_cam_0, [0, 0, 0, 1]])

        # Apply rectification and projection
        T_velo_cam = cam_calib['P_rect'] @ cam_calib['R_rect'] @ T_velo_cam_0

        return T_velo_cam[:3, :]

In [13]:
class ObjectDetector:
    """Handles object detection using YOLO."""

    def __init__(self, config: ConfigManager):
        self.config = config
        self.model = self._load_model()
        self._configure_model()

    def _load_model(self):
        """Load YOLO model."""
        model_name = self.config.get('detection.model_name', 'yolov5s')
        try:
            model = torch.hub.load('ultralytics/yolov5', model_name, pretrained=True)
            logger.info(f"Loaded model: {model_name}")
            return model
        except Exception as e:
            logger.error(f"Failed to load model {model_name}: {e}")
            raise

    def _configure_model(self):
        """Configure model parameters."""
        self.model.conf = self.config.get('detection.confidence_threshold', 0.25)
        self.model.iou = self.config.get('detection.iou_threshold', 0.25)
        self.model.max_det = self.config.get('detection.max_detections', 100)

        # Use GPU if available and configured
        if self.config.get('performance.use_gpu', True) and torch.cuda.is_available():
            self.model.cuda()
            logger.info("Using GPU acceleration")

    def detect(self, image: np.ndarray) -> List[Detection]:
        """Perform object detection on image."""
        try:
            with torch.no_grad():
                results = self.model(image)

            detections = []
            for detection in results.xyxy[0].cpu().numpy():
                x1, y1, x2, y2, conf, class_id = detection
                class_name = self.model.names[int(class_id)]

                detections.append(Detection(
                    bbox=np.array([x1, y1, x2, y2]),
                    confidence=float(conf),
                    class_id=int(class_id),
                    class_name=class_name
                ))

            logger.debug(f"Detected {len(detections)} objects")
            return detections

        except Exception as e:
            logger.error(f"Detection failed: {e}")
            return []



In [14]:
class LiDARProcessor:
    """Processes LiDAR point clouds."""

    def __init__(self, config: ConfigManager):
        self.config = config
        self.max_distance = config.get('fusion.max_distance_threshold', 100.0)

    def filter_points(self, points: np.ndarray, remove_ground: bool = True) -> np.ndarray:
        """Filter LiDAR points based on distance and ground removal."""
        # Remove points with zero intensity (common filtering)
        if points.shape[1] > 3:
            points = points[points[:, 3] > 0]

        # Distance filtering
        distances = np.linalg.norm(points[:, :3], axis=1)
        points = points[distances <= self.max_distance]

        # Ground removal using simple height threshold
        if remove_ground:
            points = points[points[:, 2] > -1.5]  # Remove points below -1.5m

        logger.debug(f"Filtered to {len(points)} LiDAR points")
        return points

    def project_to_image(self, points: np.ndarray, T_velo_cam: np.ndarray,
                        image_shape: Tuple[int, int]) -> np.ndarray:
        """Project 3D LiDAR points to 2D image coordinates."""
        # Convert to homogeneous coordinates
        points_hom = np.hstack([points[:, :3], np.ones((len(points), 1))])

        # Transform to camera coordinates
        points_cam = (T_velo_cam @ points_hom.T).T

        # Remove points behind camera
        valid_mask = points_cam[:, 2] > 0
        points_cam = points_cam[valid_mask]

        # Project to image plane
        u = points_cam[:, 0] / points_cam[:, 2]
        v = points_cam[:, 1] / points_cam[:, 2]

        # Filter points within image bounds
        h, w = image_shape[:2]
        valid_mask = (u >= 0) & (u < w) & (v >= 0) & (v < h)

        points_proj = np.column_stack([u[valid_mask], v[valid_mask], points_cam[valid_mask, 2]])

        logger.debug(f"Projected {len(points_proj)} points to image")
        return points_proj

    def cluster_points(self, points: np.ndarray) -> List[np.ndarray]:
        """Cluster LiDAR points using DBSCAN."""
        if len(points) < 3:
            return [points] if len(points) > 0 else []

        clustering = DBSCAN(
            eps=self.config.get('fusion.point_cluster_eps', 0.5),
            min_samples=self.config.get('fusion.min_cluster_size', 3)
        )

        cluster_labels = clustering.fit_predict(points[:, :2])  # Cluster based on x, y

        clusters = []
        for label in set(cluster_labels):
            if label != -1:  # Ignore noise points
                cluster_points = points[cluster_labels == label]
                clusters.append(cluster_points)

        logger.debug(f"Found {len(clusters)} point clusters")
        return clusters

In [15]:
class FusionEngine:
    """Core fusion engine that combines camera and LiDAR data."""

    def __init__(self, config: ConfigManager, calibration: CalibrationData):
        self.config = config
        self.calibration = calibration
        self.lidar_processor = LiDARProcessor(config)

    def fuse_detections(self, detections: List[Detection], lidar_points: np.ndarray,
                       image_shape: Tuple[int, int]) -> List[Detection]:
        """Fuse object detections with LiDAR data."""
        if not detections:
            return detections

        # Filter and project LiDAR points
        filtered_points = self.lidar_processor.filter_points(lidar_points)
        projected_points = self.lidar_processor.project_to_image(
            filtered_points, self.calibration.T_velo_cam, image_shape
        )

        if len(projected_points) == 0:
            logger.warning("No LiDAR points projected to image")
            return detections

        # Associate LiDAR points with detections
        fused_detections = []
        for detection in detections:
            fused_detection = self._associate_lidar_with_detection(
                detection, projected_points, filtered_points
            )
            fused_detections.append(fused_detection)

        return fused_detections

    def _associate_lidar_with_detection(self, detection: Detection,
                                       projected_points: np.ndarray,
                                       world_points: np.ndarray) -> Detection:
        """Associate LiDAR points with a single detection."""
        x1, y1, x2, y2 = detection.bbox

        # Find points within bounding box
        mask = ((projected_points[:, 0] >= x1) & (projected_points[:, 0] <= x2) &
                (projected_points[:, 1] >= y1) & (projected_points[:, 1] <= y2))

        if not np.any(mask):
            # If no points in bbox, find closest point to center
            center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
            distances = np.sqrt((projected_points[:, 0] - center_x)**2 +
                              (projected_points[:, 1] - center_y)**2)
            closest_idx = np.argmin(distances)

            detection.center_3d = world_points[closest_idx, :3]
            detection.distance = projected_points[closest_idx, 2]
        else:
            # Use median depth of points in bounding box for robustness
            bbox_points = projected_points[mask]
            bbox_world_points = world_points[mask]

            median_depth_idx = np.argsort(bbox_points[:, 2])[len(bbox_points) // 2]
            detection.center_3d = bbox_world_points[median_depth_idx, :3]
            detection.distance = bbox_points[median_depth_idx, 2]

        return detection

    def transform_to_global(self, detections: List[Detection],
                           gps_data: Dict[str, float]) -> List[Detection]:
        """Transform detection coordinates to global GPS coordinates."""
        lat0, lon0, alt0 = gps_data['lat'], gps_data['lon'], gps_data['alt']
        heading0 = gps_data['yaw']

        for detection in detections:
            if detection.center_3d is not None:
                x, y, z = detection.center_3d

                # Convert to range, azimuth, elevation
                rng = np.sqrt(x**2 + y**2 + z**2)
                az = np.degrees(np.arctan2(y, x)) + np.degrees(heading0)
                el = np.degrees(np.arctan2(np.sqrt(x**2 + y**2), z)) + 90

                # Convert to geodetic coordinates
                lat, lon, alt = pm.aer2geodetic(az, el, rng, lat0, lon0, alt0)
                detection.gps_coords = (lat, lon, alt)

        return detections

In [16]:
class FusionEngine:
    """Core fusion engine that combines camera and LiDAR data."""

    def __init__(self, config: ConfigManager, calibration: CalibrationData):
        self.config = config
        self.calibration = calibration
        self.lidar_processor = LiDARProcessor(config)

    def fuse_detections(self, detections: List[Detection], lidar_points: np.ndarray,
                       image_shape: Tuple[int, int]) -> List[Detection]:
        """Fuse object detections with LiDAR data."""
        if not detections:
            return detections

        # Filter and project LiDAR points
        filtered_points = self.lidar_processor.filter_points(lidar_points)
        projected_points = self.lidar_processor.project_to_image(
            filtered_points, self.calibration.T_velo_cam, image_shape
        )

        if len(projected_points) == 0:
            logger.warning("No LiDAR points projected to image")
            return detections

        # Associate LiDAR points with detections
        fused_detections = []
        for detection in detections:
            fused_detection = self._associate_lidar_with_detection(
                detection, projected_points, filtered_points
            )
            fused_detections.append(fused_detection)

        return fused_detections

    def _associate_lidar_with_detection(self, detection: Detection,
                                       projected_points: np.ndarray,
                                       world_points: np.ndarray) -> Detection:
        """Associate LiDAR points with a single detection."""
        x1, y1, x2, y2 = detection.bbox

        # Find points within bounding box
        mask = ((projected_points[:, 0] >= x1) & (projected_points[:, 0] <= x2) &
                (projected_points[:, 1] >= y1) & (projected_points[:, 1] <= y2))

        if not np.any(mask):
            # If no points in bbox, find closest point to center
            center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
            distances = np.sqrt((projected_points[:, 0] - center_x)**2 +
                              (projected_points[:, 1] - center_y)**2)
            closest_idx = np.argmin(distances)

            detection.center_3d = world_points[closest_idx, :3]
            detection.distance = projected_points[closest_idx, 2]
        else:
            # Use median depth of points in bounding box for robustness
            bbox_points = projected_points[mask]
            bbox_world_points = world_points[mask]

            median_depth_idx = np.argsort(bbox_points[:, 2])[len(bbox_points) // 2]
            detection.center_3d = bbox_world_points[median_depth_idx, :3]
            detection.distance = bbox_points[median_depth_idx, 2]

        return detection

    def transform_to_global(self, detections: List[Detection],
                           gps_data: Dict[str, float]) -> List[Detection]:
        """Transform detection coordinates to global GPS coordinates."""
        lat0, lon0, alt0 = gps_data['lat'], gps_data['lon'], gps_data['alt']
        heading0 = gps_data['yaw']

        for detection in detections:
            if detection.center_3d is not None:
                x, y, z = detection.center_3d

                # Convert to range, azimuth, elevation
                rng = np.sqrt(x**2 + y**2 + z**2)
                az = np.degrees(np.arctan2(y, x)) + np.degrees(heading0)
                el = np.degrees(np.arctan2(np.sqrt(x**2 + y**2), z)) + 90

                # Convert to geodetic coordinates
                lat, lon, alt = pm.aer2geodetic(az, el, rng, lat0, lon0, alt0)
                detection.gps_coords = (lat, lon, alt)

        return detections

In [17]:
class VisualizationEngine:
    """Handles visualization of fusion results."""

    @staticmethod
    def draw_detections(image: np.ndarray, detections: List[Detection]) -> np.ndarray:
        """Draw detections on image."""
        vis_image = image.copy()

        for detection in detections:
            x1, y1, x2, y2 = detection.bbox.astype(int)

            # Draw bounding box
            color = (0, 255, 0) if detection.distance else (255, 0, 0)
            cv2.rectangle(vis_image, (x1, y1), (x2, y2), color, 2)

            # Draw label
            label = f"{detection.class_name}: {detection.confidence:.2f}"
            if detection.distance:
                label += f" ({detection.distance:.1f}m)"

            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            cv2.rectangle(vis_image, (x1, y1 - label_size[1] - 10),
                         (x1 + label_size[0], y1), color, -1)
            cv2.putText(vis_image, label, (x1, y1 - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        return vis_image

    @staticmethod
    def draw_lidar_overlay(image: np.ndarray, projected_points: np.ndarray) -> np.ndarray:
        """Draw LiDAR points overlay on image."""
        vis_image = image.copy()

        for point in projected_points:
            u, v, depth = point
            if depth > 0:
                # Color code by depth
                color_intensity = min(255, int(255 * (1.0 - depth / 50.0)))
                color = (color_intensity, 0, 255 - color_intensity)
                cv2.circle(vis_image, (int(u), int(v)), 2, color, -1)

        return vis_image

In [18]:
class FusionSystem:
    """Main fusion system orchestrator."""

    def __init__(self, config_path: Optional[str] = None):
        self.config = ConfigManager(config_path)
        self.performance_monitor = PerformanceMonitor()

        # Initialize components
        self.detector = None
        self.fusion_engine = None
        self.data_loader = None
        self.calibration = None

        logger.info("LiDAR-Camera Fusion System initialized")

    def initialize(self, data_path: str, calib_path: str):
        """Initialize system with data and calibration paths."""
        try:
            # Load calibration data
            calib_loader = CalibrationLoader(calib_path)
            self.calibration = calib_loader.load_calibration()

            # Initialize data loader
            self.data_loader = DataLoader(data_path)

            # Initialize detector
            self.detector = ObjectDetector(self.config)

            # Initialize fusion engine
            self.fusion_engine = FusionEngine(self.config, self.calibration)

            logger.info("System initialization completed successfully")

        except Exception as e:
            logger.error(f"System initialization failed: {e}")
            raise

    def process_frame(self, frame_idx: int) -> Tuple[List[Detection], np.ndarray]:
        """Process a single frame through the complete pipeline."""
        with self.performance_monitor.measure_time("total_frame_processing"):
            # Load sensor data
            with self.performance_monitor.measure_time("data_loading"):
                sensor_frame = self.data_loader.load_frame(frame_idx)

            # Object detection
            with self.performance_monitor.measure_time("object_detection"):
                detections = self.detector.detect(sensor_frame.image)

            # LiDAR-Camera fusion
            with self.performance_monitor.measure_time("sensor_fusion"):
                fused_detections = self.fusion_engine.fuse_detections(
                    detections, sensor_frame.lidar_points, sensor_frame.image.shape
                )

            # Global coordinate transformation
            with self.performance_monitor.measure_time("coordinate_transformation"):
                final_detections = self.fusion_engine.transform_to_global(
                    fused_detections, sensor_frame.gps_data
                )

            # Visualization
            with self.performance_monitor.measure_time("visualization"):
                vis_image = VisualizationEngine.draw_detections(
                    sensor_frame.image, final_detections
                )

            # Log metrics
            self.performance_monitor.log_detection_count(len(final_detections))

            return final_detections, vis_image

    def process_sequence(self, start_frame: int = 0, num_frames: int = 10) -> Dict[str, Any]:
        """Process a sequence of frames."""
        results = []

        logger.info(f"Processing {num_frames} frames starting from frame {start_frame}")

        for i in range(start_frame, min(start_frame + num_frames, self.data_loader.num_frames)):
            try:
                detections, vis_image = self.process_frame(i)
                results.append({
                    'frame_id': i,
                    'detections': detections,
                    'visualization': vis_image,
                    'timestamp': time.time()
                })

                if i % 10 == 0:
                    logger.info(f"Processed frame {i}/{start_frame + num_frames - 1}")

            except Exception as e:
                logger.error(f"Failed to process frame {i}: {e}")
                continue

        # Generate summary report
        summary = self._generate_sequence_summary(results)
        logger.info(f"Sequence processing completed. {len(results)} frames processed successfully")

        return {
            'results': results,
            'summary': summary,
            'performance_report': self.performance_monitor.get_performance_report()
        }

    def _generate_sequence_summary(self, results: List[Dict]) -> Dict[str, Any]:
        """Generate summary statistics for processed sequence."""
        if not results:
            return {}

        total_detections = sum(len(r['detections']) for r in results)
        detection_counts = [len(r['detections']) for r in results]

        # Count detections by class
        class_counts = {}
        distance_stats = []

        for result in results:
            for detection in result['detections']:
                class_counts[detection.class_name] = class_counts.get(detection.class_name, 0) + 1
                if detection.distance:
                    distance_stats.append(detection.distance)

        return {
            'total_frames': len(results),
            'total_detections': total_detections,
            'avg_detections_per_frame': np.mean(detection_counts),
            'detection_by_class': class_counts,
            'distance_statistics': {
                'mean_distance': np.mean(distance_stats) if distance_stats else 0,
                'median_distance': np.median(distance_stats) if distance_stats else 0,
                'max_distance': np.max(distance_stats) if distance_stats else 0,
                'min_distance': np.min(distance_stats) if distance_stats else 0
            }
        }

    def run_real_time_simulation(self, fps: float = 10.0):
        """Simulate real-time processing."""
        frame_interval = 1.0 / fps

        logger.info(f"Starting real-time simulation at {fps} FPS")

        try:
            for frame_idx in range(self.data_loader.num_frames):
                start_time = time.time()

                # Process frame
                detections, vis_image = self.process_frame(frame_idx)

                # Display results (in real implementation, this would go to display/dashboard)
                print(f"Frame {frame_idx}: {len(detections)} detections")
                for det in detections[:3]:  # Show first 3 detections
                    print(f"  {det.class_name}: {det.confidence:.2f}, "
                          f"distance: {det.distance:.1f}m" if det.distance else "distance: N/A")

                # Maintain frame rate
                processing_time = time.time() - start_time
                sleep_time = max(0, frame_interval - processing_time)

                if sleep_time > 0:
                    time.sleep(sleep_time)
                else:
                    logger.warning(f"Frame {frame_idx} processing exceeded target time: "
                                 f"{processing_time:.3f}s > {frame_interval:.3f}s")

        except KeyboardInterrupt:
            logger.info("Real-time simulation stopped by user")
        except Exception as e:
            logger.error(f"Real-time simulation failed: {e}")

    def benchmark_performance(self, num_iterations: int = 100) -> Dict[str, Any]:
        """Run performance benchmarks."""
        logger.info(f"Running performance benchmark with {num_iterations} iterations")

        # Warm up
        for _ in range(5):
            self.process_frame(0)

        # Benchmark
        benchmark_results = []
        for i in range(num_iterations):
            frame_idx = i % self.data_loader.num_frames
            start_time = time.time()

            detections, _ = self.process_frame(frame_idx)

            processing_time = time.time() - start_time
            benchmark_results.append({
                'iteration': i,
                'frame_idx': frame_idx,
                'processing_time': processing_time,
                'num_detections': len(detections)
            })

        # Analyze results
        processing_times = [r['processing_time'] for r in benchmark_results]

        return {
            'num_iterations': num_iterations,
            'mean_processing_time': np.mean(processing_times),
            'std_processing_time': np.std(processing_times),
            'min_processing_time': np.min(processing_times),
            'max_processing_time': np.max(processing_times),
            'fps_capability': 1.0 / np.mean(processing_times),
            'percentiles': {
                '50th': np.percentile(processing_times, 50),
                '95th': np.percentile(processing_times, 95),
                '99th': np.percentile(processing_times, 99)
            }
        }

    def save_results(self, results: Dict[str, Any], output_dir: str):
        """Save processing results to disk."""
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        # Save summary as JSON
        summary_path = output_path / 'processing_summary.json'
        with open(summary_path, 'w') as f:
            # Convert numpy types to Python types for JSON serialization
            json_summary = self._convert_numpy_types(results['summary'])
            json.dump(json_summary, f, indent=2)

        # Save performance report
        perf_path = output_path / 'performance_report.json'
        with open(perf_path, 'w') as f:
            perf_report = self._convert_numpy_types(results['performance_report'])
            json.dump(perf_report, f, indent=2)

        # Save detection results as CSV
        if 'results' in results:
            detections_data = []
            for result in results['results']:
                for detection in result['detections']:
                    row = {
                        'frame_id': result['frame_id'],
                        'class_name': detection.class_name,
                        'confidence': detection.confidence,
                        'bbox_x1': detection.bbox[0],
                        'bbox_y1': detection.bbox[1],
                        'bbox_x2': detection.bbox[2],
                        'bbox_y2': detection.bbox[3],
                        'distance': detection.distance,
                        'timestamp': detection.timestamp
                    }

                    if detection.center_3d is not None:
                        row.update({
                            'center_3d_x': detection.center_3d[0],
                            'center_3d_y': detection.center_3d[1],
                            'center_3d_z': detection.center_3d[2]
                        })

                    if detection.gps_coords is not None:
                        row.update({
                            'gps_lat': detection.gps_coords[0],
                            'gps_lon': detection.gps_coords[1],
                            'gps_alt': detection.gps_coords[2]
                        })

                    detections_data.append(row)

            df = pd.DataFrame(detections_data)
            df.to_csv(output_path / 'detections.csv', index=False)

        logger.info(f"Results saved to {output_path}")

    def _convert_numpy_types(self, obj):
        """Convert numpy types to Python types for JSON serialization."""
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: self._convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self._convert_numpy_types(item) for item in obj]
        else:
            return obj

In [19]:
# Example usage and testing
def main():
    """Main function demonstrating system usage."""

    # Configuration
    config = {
        'detection': {
            'model_name': 'yolov5s',
            'confidence_threshold': 0.3,
            'iou_threshold': 0.25
        },
        'fusion': {
            'max_distance_threshold': 80.0,
            'point_cluster_eps': 0.3,
            'min_cluster_size': 5
        },
        'performance': {
            'use_gpu': True,
            'num_threads': 4
        }
    }

    # Save example config
    with open('fusion_config.yaml', 'w') as f:
        yaml.dump(config, f)

    # Initialize system
    fusion_system = FusionSystem('fusion_config.yaml')

    # Example data paths (adjust for your setup)
    data_path = "2011_10_03/2011_10_03_drive_0047_sync"
    calib_path = "2011_10_03"

    try:
        # Initialize with data
        fusion_system.initialize(data_path, calib_path)

        # Process a sequence of frames
        results = fusion_system.process_sequence(start_frame=0, num_frames=20)

        # Save results
        fusion_system.save_results(results, 'output')

        # Run performance benchmark
        benchmark_results = fusion_system.benchmark_performance(num_iterations=50)
        print("\nPerformance Benchmark Results:")
        print(f"Mean processing time: {benchmark_results['mean_processing_time']:.4f}s")
        print(f"FPS capability: {benchmark_results['fps_capability']:.1f}")
        print(f"95th percentile: {benchmark_results['percentiles']['95th']:.4f}s")

        # Demonstrate real-time simulation (uncomment to run)
        # fusion_system.run_real_time_simulation(fps=5.0)

    except Exception as e:
        logger.error(f"System execution failed: {e}")
        return 1

    return 0

In [20]:

# Unit tests
class TestFusionSystem:
    """Unit tests for the fusion system."""

    def test_calibration_loading(self):
        """Test calibration data loading and validation."""
        # Mock calibration data
        mock_calib = CalibrationData(
            P_rect=np.eye(3, 4),
            R_rect=np.eye(4),
            T_cam_velo=np.eye(4),
            T_velo_cam=np.eye(3, 4),
            T_cam_imu=np.eye(4),
            camera_intrinsics={'fx': 700, 'fy': 700, 'cx': 320, 'cy': 240},
            distortion_coeffs=np.zeros(5)
        )

        assert mock_calib.P_rect.shape == (3, 4)
        assert mock_calib.R_rect.shape == (4, 4)
        print("✓ Calibration loading test passed")

    def test_config_management(self):
        """Test configuration management."""
        config = ConfigManager()

        # Test default values
        assert config.get('detection.confidence_threshold') == 0.25
        assert config.get('fusion.max_distance_threshold') == 100.0

        # Test nested access
        assert config.get('detection.model_name') == 'yolov5s'
        print("✓ Configuration management test passed")

    def test_performance_monitor(self):
        """Test performance monitoring."""
        monitor = PerformanceMonitor()

        with monitor.measure_time("test_operation"):
            time.sleep(0.1)

        report = monitor.get_performance_report()
        assert report['total_frames_processed'] == 0
        assert len(monitor.metrics['processing_times']) == 1
        print("✓ Performance monitoring test passed")


if __name__ == "__main__":
    # Run tests
    test_suite = TestFusionSystem()
    test_suite.test_calibration_loading()
    test_suite.test_config_management()
    test_suite.test_performance_monitor()

    print("\nAll tests passed! ✓")
    print("\nStarting main system...")

    # Run main system
    exit_code = main()

ERROR:__main__:Failed to load calibration data: [Errno 2] No such file or directory: '2011_10_03/calib_cam_to_cam.txt'
ERROR:__main__:System initialization failed: [Errno 2] No such file or directory: '2011_10_03/calib_cam_to_cam.txt'
ERROR:__main__:System execution failed: [Errno 2] No such file or directory: '2011_10_03/calib_cam_to_cam.txt'


✓ Calibration loading test passed
✓ Configuration management test passed
✓ Performance monitoring test passed

All tests passed! ✓

Starting main system...
